We've included 2 solutions here. There are also many other ways of formatting this data. Don't wory if you haven't done it exactly like we have.... this is just guide and not a bible.

In [ ]:
import pandas as pd
import requests
from datetime import datetime
import pytz

# Solution using a for loop

First we need to take a look at the JSON so we know what we're dealing with.
- We can view what the JSON contains by checking out the [documentation on the OWM website](https://openweathermap.org/forecast5)
- Or this can be done manually, like below:

In [ ]:
city = 'Berlin'
API_key = 'YOUR_API_KEY_HERE'

# check out the docs for more info on making an api call https://openweathermap.org/forecast5
url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")

response = requests.get(url)
json = response.json()

json

Now we've discovered what information we have to work with. Let's decide what we want to keep and what we wish to lose.

I feel that from json['list'] it would be good to keep
- 'weather.main', 'weather.description', 'dt_txt', 'main.temp', 'main.feels_like' 'clouds.all', 'rain.3h', 'snow.3h' 'wind.speed', 'wind.deg', 'main.humidity', 'main.pressure'. 

And from json['city'] it would be good to keep 
- 'name' and 'country. 

Just to make sure that we got the right place. And as an added extra we'll also include the time that API call was made, so we know how up to date our forecast is.

**Optional:** let's get a timestamp of when we get the data. Datetime uses the uses the current time of the system, which on local computers is normally correct. But as we're in the cloud, computers are not always in our country, and we therefore add on the timezone module to ensure that our timestamp is local to us and not the computer.

In [ ]:
tz = pytz.timezone('Europe/Berlin')
now = datetime.now().astimezone(tz)

now

datetime.datetime(2022, 7, 19, 10, 18, 27, 111572, tzinfo=<DstTzInfo 'Europe/Berlin' CEST+2:00:00 DST>)

**Next** let's loop through the json['list'] information get the weather information

In [ ]:
# we'll store the information in this dicitonary:
weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': []}
                #'information_retrieved_at': []}

# let's begin the loop
for i in json['list']:
  weather_dict['city'].append(json['city']['name'])
  weather_dict['country'].append(json['city']['country'])
  weather_dict['forecast_time'].append(i['dt_txt'])
  weather_dict['outlook'].append(i['weather'][0]['main'])
  weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
  weather_dict['temperature'].append(i['main']['temp'])
  weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
  weather_dict['clouds'].append(i['clouds']['all'])
  # sometimes the data is missing for rain and snow. As it is not always raining or snowing
  # we cannot make a DataFrame unless the lists are all the same length, therefore missing values are bad
  # here we say try to append a value if there is one. If not, append a 0
  try:
      weather_dict['rain'].append(i['rain']['3h'])
  except:
      weather_dict['rain'].append('0')
  try:
      weather_dict['snow'].append(i['snow']['3h'])
  except:
      weather_dict['snow'].append('0')
  weather_dict['wind_speed'].append(i['wind']['speed'])
  weather_dict['wind_deg'].append(i['wind']['deg'])
  weather_dict['humidity'].append(i['main']['humidity'])
  weather_dict['pressure'].append(i['main']['pressure'])
  #weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))
  

**Now** we convert our dictionary to a DataFrame

In [ ]:
weather_from_dict_df = pd.DataFrame(weather_dict)

weather_from_dict_df.head()

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure
0,Berlin,DE,2022-07-06 15:00:00,Clouds,scattered clouds,20.21,19.54,32,0,0,5.16,268,48,1019
1,Berlin,DE,2022-07-06 18:00:00,Clouds,broken clouds,19.28,18.52,61,0,0,3.38,282,48,1020
2,Berlin,DE,2022-07-06 21:00:00,Clouds,overcast clouds,16.86,15.99,87,0,0,3.47,247,53,1019
3,Berlin,DE,2022-07-07 00:00:00,Clouds,broken clouds,15.03,14.13,81,0,0,2.54,236,59,1018
4,Berlin,DE,2022-07-07 03:00:00,Rain,light rain,12.98,12.61,100,1.02,0,3.80,220,87,1016


**As a final step**, to keep everything tidy: let's bring everything we did together in a function. And allow the function to take a list of cities as an input

In [ ]:
def get_weather_loop(cities):

  API_key = 'YOUR_API_KEY_HERE'

  tz = pytz.timezone('Europe/Berlin')
  now = datetime.now().astimezone(tz)

  weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': [],
                'information_retrieved_at': []}

  for city in cities:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()

    for i in json['list']:
      weather_dict['city'].append(json['city']['name'])
      weather_dict['country'].append(json['city']['country'])
      weather_dict['forecast_time'].append(i['dt_txt'])
      weather_dict['outlook'].append(i['weather'][0]['main'])
      weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
      weather_dict['temperature'].append(i['main']['temp'])
      weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
      weather_dict['clouds'].append(i['clouds']['all'])
      try:
          weather_dict['rain'].append(i['rain']['3h'])
      except:
          weather_dict['rain'].append('0')
      try:
          weather_dict['snow'].append(i['snow']['3h'])
      except:
          weather_dict['snow'].append('0')
      weather_dict['wind_speed'].append(i['wind']['speed'])
      weather_dict['wind_deg'].append(i['wind']['deg'])
      weather_dict['humidity'].append(i['main']['humidity'])
      weather_dict['pressure'].append(i['main']['pressure'])
      weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))

  return pd.DataFrame(weather_dict)

In [ ]:
get_weather_loop(['Berlin', 'London'])

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,2022-07-06 15:00:00,Clouds,scattered clouds,20.21,19.54,32,0,0,5.16,268,48,1019,06/07/2022 14:45:03
1,Berlin,DE,2022-07-06 18:00:00,Clouds,broken clouds,19.28,18.52,61,0,0,3.38,282,48,1020,06/07/2022 14:45:03
2,Berlin,DE,2022-07-06 21:00:00,Clouds,overcast clouds,16.86,15.99,87,0,0,3.47,247,53,1019,06/07/2022 14:45:03
3,Berlin,DE,2022-07-07 00:00:00,Clouds,broken clouds,15.03,14.13,81,0,0,2.54,236,59,1018,06/07/2022 14:45:03
4,Berlin,DE,2022-07-07 03:00:00,Rain,light rain,12.98,12.61,100,1.02,0,3.80,220,87,1016,06/07/2022 14:45:03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,London,GB,2022-07-11 00:00:00,Clouds,scattered clouds,18.52,17.99,34,0,0,2.71,114,60,1025,06/07/2022 14:45:03
76,London,GB,2022-07-11 03:00:00,Clear,clear sky,17.27,16.85,0,0,0,1.62,103,69,1024,06/07/2022 14:45:03
77,London,GB,2022-07-11 06:00:00,Clear,clear sky,18.59,18.10,0,0,0,2.01,107,61,1025,06/07/2022 14:45:03
78,London,GB,2022-07-11 09:00:00,Clear,clear sky,24.31,23.81,2,0,0,2.09,113,39,1024,06/07/2022 14:45:03


# Solution using `.json_normalize()`

**First** we need to make an API call to get our information

In [ ]:
city = 'Berlin'
API_key = 'YOUR_API_KEY_HERE'
url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric") 
response = requests.get(url)
json = response.json()

If we now try to use `.json_normalize()` on the `list` part of the json, you'll see that json_normalize can get confused when it hits a list instead of a subdictionary. Take a look at the weather column below

In [ ]:
pd.DataFrame((json['list'])).head()

,dt,main,weather,clouds,wind,visibility,pop,sys,dt_txt,rain
0,1657119600,"{'temp': 20.21, 'feels_like': 19.54, 'temp_min...","[{'id': 802, 'main': 'Clouds', 'description': ...",{'all': 32},"{'speed': 5.16, 'deg': 268, 'gust': 6.9}",10000,0.00,{'pod': 'd'},2022-07-06 15:00:00,NaN
1,1657130400,"{'temp': 19.28, 'feels_like': 18.52, 'temp_min...","[{'id': 803, 'main': 'Clouds', 'description': ...",{'all': 61},"{'speed': 3.38, 'deg': 282, 'gust': 5.84}",10000,0.00,{'pod': 'd'},2022-07-06 18:00:00,NaN
2,1657141200,"{'temp': 16.86, 'feels_like': 15.99, 'temp_min...","[{'id': 804, 'main': 'Clouds', 'description': ...",{'all': 87},"{'speed': 3.47, 'deg': 247, 'gust': 8.08}",10000,0.00,{'pod': 'n'},2022-07-06 21:00:00,NaN
3,1657152000,"{'temp': 15.03, 'feels_like': 14.13, 'temp_min...","[{'id': 803, 'main': 'Clouds', 'description': ...",{'all': 81},"{'speed': 2.54, 'deg': 236, 'gust': 5.32}",10000,0.00,{'pod': 'n'},2022-07-07 00:00:00,NaN
4,1657162800,"{'temp': 12.98, 'feels_like': 12.61, 'temp_min...","[{'id': 500, 'main': 'Rain', 'description': 'l...",{'all': 100},"{'speed': 3.8, 'deg': 220, 'gust': 9.45}",10000,0.98,{'pod': 'd'},2022-07-07 03:00:00,{'3h': 1.02}


To fix this we need to flatten the nested list using the parameter `record_path`, and then select the other columns we'd like using the parameter `meta`. At the end of the json_normalize parameters we have `errors='ignore'`, which means that if ever there is a missing value in the JSON, a `NaN` gets inserted into the table. If you'd like more information, please check out the [docs](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html)

In [ ]:
json_norm_df = pd.json_normalize(json['list'], 
                                record_path=['weather'], 
                                meta=['dt_txt', ['main', 'temp'], ['main', 'feels_like'], ['clouds', 'all'], ['rain', '3h'], ['snow', '3h'], ['wind', 'speed'], ['wind', 'deg'], ['main', 'humidity'], ['main', 'pressure']],
                                errors='ignore')

json_norm_df.head()

,id,main,description,icon,dt_txt,main.temp,main.feels_like,clouds.all,rain.3h,snow.3h,wind.speed,wind.deg,main.humidity,main.pressure
0,802,Clouds,scattered clouds,03d,2022-07-06 15:00:00,20.21,19.54,32,NaN,NaN,5.16,268,48,1019
1,803,Clouds,broken clouds,04d,2022-07-06 18:00:00,19.28,18.52,61,NaN,NaN,3.38,282,48,1020
2,804,Clouds,overcast clouds,04n,2022-07-06 21:00:00,16.86,15.99,87,NaN,NaN,3.47,247,53,1019
3,803,Clouds,broken clouds,04n,2022-07-07 00:00:00,15.03,14.13,81,NaN,NaN,2.54,236,59,1018
4,500,Rain,light rain,10d,2022-07-07 03:00:00,12.98,12.61,100,1.02,NaN,3.8,220,87,1016


Now let's tidy this DataFrame up by
- dropping some columns
- renaming some columns
- adding some columns

In [ ]:
# drop id and icon
json_norm_df = json_norm_df.drop(columns=['id', 'icon']).copy()

In [ ]:
# rename some columns
json_norm_df.rename(columns={'main': 'outlook',
                             'description': 'detailed_outlook',
                             'dt_txt': 'forecast_time',
                             'main.temp': 'temperature',
                             'main.feels_like': 'temperature_feels_like',
                             'clouds.all': 'clouds',
                             'rain.3h': 'rain',
                             'snow.3h': 'snow',
                             'wind.speed': 'wind_speed',
                             'wind.deg': 'wind_deg',
                             'main.humidity': 'humidity',
                             'main.pressure': 'pressure',},
                    inplace=True)

In [ ]:
# add some columns
json_norm_df.insert(0, 'city', json['city']['name'])
json_norm_df.insert(1, 'country', json['city']['country'])
#json_norm_df['information_retrieved_at'] = now.strftime("%d/%m/%Y %H:%M:%S")

# rearrange the columns
json_norm_df = json_norm_df[['city', 'country', 'forecast_time', 'outlook', 'detailed_outlook',
       'temperature', 'temperature_feels_like', 'clouds', 'rain', 'snow',
       'wind_speed', 'wind_deg', 'humidity', 'pressure',
       'information_retrieved_at']]

json_norm_df.head()

KeyError: ignored

Now we have a very similar DataFrame to what we had with the for loop. Again, let's put this in a function that can take a list of cities

In [ ]:
def get_weather_norm(cities):
  
  API_key = 'YOUR_API_KEY_HERE'

  df_list = []

  for city in cities:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric") 
    response = requests.get(url)
    json = response.json()

    json_norm_df = pd.json_normalize(json['list'], 
                                record_path=['weather'], 
                                meta=['dt_txt', ['main', 'temp'], ['main', 'feels_like'], ['clouds', 'all'], ['rain', '3h'], ['snow', '3h'], ['wind', 'speed'], ['wind', 'deg'], ['main', 'humidity'], ['main', 'pressure']], 
                                errors='ignore')
    json_norm_df.drop(columns=['id', 'icon'], inplace=True)
    json_norm_df.rename(columns={'main': 'outlook',
                             'description': 'detailed_outlook',
                             'dt_txt': 'forecast_time',
                             'main.temp': 'temperature',
                             'main.feels_like': 'temperature_feels_like',
                             'clouds.all': 'clouds',
                             'rain.3h': 'rain',
                             'snow.3h': 'snow',
                             'wind.speed': 'wind_speed',
                             'wind.deg': 'wind_deg',
                             'main.humidity': 'humidity',
                             'main.pressure': 'pressure',},
                    inplace=True)
    json_norm_df.insert(0, 'city', json['city']['name'])
    json_norm_df.insert(1, 'country', json['city']['country'])
    json_norm_df['information_retrieved_at'] = now.strftime("%d/%m/%Y %H:%M:%S")
    json_norm_df[['city', 'country', 'forecast_time', 'outlook', 'detailed_outlook',
          'temperature', 'temperature_feels_like', 'clouds', 'rain', 'snow',
          'wind_speed', 'wind_deg', 'humidity', 'pressure',
          'information_retrieved_at']]
    df_list.append(json_norm_df)
  return pd.concat(df_list, ignore_index=True)

In [ ]:
get_weather_norm(['Berlin', 'London'])